In [1]:
# A easy units converter.

class Unit:

    def __init__(self, d):
        self.d = {}
        for unit, power in d.items():
            if power: # value is not 0
                self.d.update({unit : power})

    def __repr__(self): # for commandline convenience
        if not self.d:
            return "unitless"
        terms = []
        for unit, power in self.d.items():
            if power == 1:
                term = unit
            else:
                term = '{0}**{1}'.format(unit, power)
            terms.append(term)
        return ' * '.join(terms)

    def __str__(self): # This outputs basic representation
        return str(self.d)

    def __eq__(self, other): # prevent 'same unit but are different objects'
        return self.d == other.d
        #__ne__ is defined to be negation of __eq__ in python 3

    def __mul__(self, other):
        result = self.d.copy() # prevent override
        for unit, power in other.d.items(): # each unit and its power
            result[unit] = result.get(unit, 0) + power # deal with when self doesn't have a field in other
            if result[unit] == 0: # prevent empty values
                del result[unit]
        return Unit(result)

    def __truediv__(self, other):
        result = self.d.copy() # prevent override
        for (unit, power) in other.d.items(): # each unit and its power
            result[unit] = result.get(unit, 0) - power # deal with when self doesn't have a field in other
            if result[unit] == 0: # prevent empty values
                del result[unit]
        return Unit(result)

    def __pow__(self, p): # do not suggest fraction power
        result = {}
        for (unit, power) in self.d.items():
            result.update({unit:power * p})
        return Unit(result)

In [2]:
# # A easy physical quantity calculator.

class Quantity:
    
    def __init__(self, value, unit):
        self.value = value
        self.unit = unit
   
    def __repr__(self):
        return '{0} {1}'.format(self.value, repr(self.unit))
    
    def __str__(self):
        return '({0}, {1})'.format(self.value, self.unit)
    
    def __contains__(self, other):
        return self.unit == other.unit
    
    def __eq__(self, other):
        return self.value == other.value and self.unit == other.unit
    
    def __add__(self, other):
        if isinstance(other, (int, float)): # handles 1 * kg / kg + 1
            return self + Quantity(other, Unit({})) # recursion
        assert self.unit == other.unit # check for same unit on addition. should i consider self.value == 0?
        return Quantity(self.value + other.value, self.unit)
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        if isinstance(other, (int, float)): # handles 1 * kg / kg - 1
            return self - Quantity(other, Unit({}))
        assert self.unit == other.unit # check for same unit on subtraction
        return Quantity(self.value - other.value, self.unit)
    
    def __neg__(self):
        return Quantity(-self.value, self.unit)
    
    def __rsub__(self, other):
        return - (self - other) # self - other calls __sub__, other - self calls __rsub__ on self again
    
    def __mul__(self, other): # both kg * 10 and kg * m works
        if isinstance(other, (int, float)):
            return self * Quantity(other, Unit({}))
        return Quantity(self.value * other.value, self.unit * other.unit)
    
    def __rmul__(self, other): # right multiplication, 10 * kg works. commutative
        return self * other
    
    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            return self / Quantity(other, Unit({}))
        return Quantity(self.value / other.value, self.unit / other.unit)
        
    def __rtruediv__(self, other):
        return (self / other) ** -1 # use **-1 to represent inverse
    
    def __pow__(self, p):
        return Quantity(self.value ** p, self.unit ** p)

In [3]:
# 7 SI units

kg = Quantity(1, Unit({'kg' : 1}))
m = Quantity(1, Unit({'m' : 1}))
s = Quantity(1, Unit({'s' : 1}))
A = Quantity(1, Unit({'A' : 1}))
mol = Quantity(1, Unit({'mol' : 1}))
K = Quantity(1, Unit({'K' : 1}))
Cd = Quantity(1, Unit({'Cd' : 1}))

In [4]:
# variables start with '_' are physical quantities names

Hz = s ** -1 # Hertz

_m = kg # mass
_v = m / s # velocity
_a = m / s**2 # acceleration
_j = m / s**3 # jerk
_p = kg * _v # momentum

_omega = 1 / s # angular velocity
_alpha = 1 / s**2 # angular acceleration
_L = kg * m * _v # angular momentum
_I = kg * m**2 # moment of inertia

_l = m # length
_A = m**2 # area
_V = m**3 # volume
_rho = kg / m**3 # density

N = kg * _a # Newton
_M = N * m # torque

Pa = N / m**2 # Pascal
J = N * m # Joule
W = J / s # Watt

C = A * s # Coulomb
_q = C # electric charge
_E = N / C # electric field
V = _E * m # Voltage
F = C / V # Farad
T = N / (C * _v) # Tesla
Wb = T * m**2 # Weber
Ohm = V / A # Ohm
H = Ohm * s # Joseph Henry

In [5]:
from math import *

In [6]:
# physicsal constants
# https://en.wikipedia.org/wiki/List_of_physical_constants

_c = 299792458 * m / s # speed of light
_h = 6.62607015E-34 * J * s # Planck constant
_hbar = _h / (2 * pi) # reduced Planck constant
_G = 6.67430E-11 * m**3 * kg**-1 * s**-2 # Newtonian constatn of gravitation
_mu_0 = 1.25663706212E-6 * N * A **-2 # vacuum magnetic ermeability
_epsilon_0 = 1 / (_mu_0 * _c**2) # vacuum electric permittivity
_e = 1.602E-19 * C # elementary charge
_N_A = 6.02214076E23 * mol**-1 # Avogadro constant
_k_B = 1.380649E-23 * J / K # Boltzmann constant
_m_e = 9.1093837015E-31 * kg # electron mass
_m_p = 1.67262192369E-27 * kg # proton mass
_m_n = 1.67492749804E-27 * kg # neutron mass
Da = 1.66053906660E-27 * kg # atomic mass constant
_R = _N_A * _k_B # ideal gas constant
_sigma = 5.670374419E-8 * W / m**2 / K**4 # Stefan-Boltzmann constant

_g = 9.81 * m / s**2 # gravitational acceleration
_k = 9E9 * N * m**2 / C**2 # Coulomb constant
_eV = _e * V # electrovolt. this is a constant, not a unit.
_keV = 1E3 * _eV # kilo electrovolt
_MeV = 1E6 * _eV # mega electrovolt
_GeV = 1E9 * _eV # giga electrovolt
_TeV = 1E12 * _eV # tetra electrovolt
_MeVpc2 = _MeV / _c**2 # mega electrovolt per c**2. this is a constant, not a unit

In [7]:
assert 1 / s in Hz
assert kg * _v**2 in J
assert N / C in V / m
assert _R in J / mol / K
assert _MeVpc2 in kg

assert 1 * m != 1 * kg
assert 10 * J == 10 * N * m
assert 2 * kg != 3 * kg